<a href="https://colab.research.google.com/github/RodolfoFerro/cimps20/blob/main/notebooks/05-Empaquetado_de_modelos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Empaquetado de modelos

> **Rodolfo Ferro** <br>
> Google Dev Expert en ML, 2020.
>
> _Redes:_
> - GitHub - [RodolfoFerro](https://github.com/RodolfoFerro)
> - Twitter - [@FerroRodolfo](https://twitter.com/FerroRodolfo)
> - Instagram - [@rodo_ferro](https://instagram.com/rodo_ferro)

## Contenidos

#### **Sección VI**
1. **Código:** Un nuevo problema
2. **Código:** Guardado de modelos en TF


## **Sección VI**

### El dataset de dígitos escritos a mano

Comencemos importando TensorFlow.

In [ ]:
import tensorflow as tf
print(tf.__version__)

Los datos de MNIST están disponibles directamente en la API de conjuntos de datos de `tf.keras` y podemos importarlos de la misma manera que hicimos ocn el de modas.

Llamar a `load_data` en este objeto nos dará dos conjuntos con los valores de entrenamiento y prueba para los gráficos que contienen las prendas y sus etiquetas.

In [ ]:
mnist = tf.keras.datasets.mnist
(training_images, training_labels), (test_images, test_labels) = mnist.load_data()

¿Cómo se ven estos valores?

Imprimamos una imagen de entrenamiento y una etiqueta de entrenamiento para ver.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
np.set_printoptions(linewidth=200)


# Set index of image to be seen
img_index = 0

# Plot image
plt.imshow(training_images[img_index], cmap='gray')
plt.axis(False)

print("Label:", training_labels[img_index])
print("Matrix:\n", training_images[img_index])

### Preparación de los datos

Notarás que todos los valores están entre 0 y 255. Si estamos entrenando una red neuronal, por varias razones es más fácil si transformamos los valores para tratar todos con valores entre 0 y 1. Este proceso se llama **normalización**.

Además, para este proceso añadiremos la expansión de dimensiones para poder alimentar a la red.

In [ ]:
training_images = training_images.reshape(60000, 28, 28, 1)
training_images = training_images / 255.0

test_images = test_images.reshape(10000, 28, 28, 1)
test_images = test_images/255.0

### Creación del modelo

Podemos hacer uso del potencial de una red neuronal convolucional. 

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPooling2D(2, 2),
    # Añadamos algunas capas convolucionales extra
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(
    optimizer=tf.optimizers.Adam(),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

### Entrenamiento del modelo

In [ ]:
model.fit(training_images, training_labels, epochs=3)

### Evaluación del modelo

In [ ]:
model.evaluate(test_images, test_labels)

### Predicción


In [ ]:
test_index = 30

plt.imshow(np.squeeze(test_images[test_index]), cmap='gray')
plt.axis(False)

print("Label:", test_labels[test_index])
prediction = model.predict(np.expand_dims(test_images[test_index], axis=0))
print("Prediction:", np.argmax(prediction))

### Guardado del modelo entrenado

Lo que haremos será guardar los pesos y la arquitectura del modelo en 2 archivos distintos.

In [ ]:
# Serialize model to JSON:
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)

# Serialize weights to HDF5 (h5py needed):
model.save_weights("model.h5")
print("Model saved to disk.")

Los mismos archivos pueden ser cargados utilizando las funciones correspondientes:

In [ ]:
# Load json and create model:
model_from_json = tf.keras.models.model_from_json

json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)

# Load weights into loaded model:
loaded_model.load_weights("model.h5")
print("Model loaded from disk.")

Ejemplo de predicción con el modelo cargado:

In [ ]:
test_index = 30

plt.imshow(np.squeeze(test_images[test_index]), cmap='gray')
plt.axis(False)

print("Label:", test_labels[test_index])
prediction = loaded_model.predict(np.expand_dims(test_images[test_index], axis=0))
print("Prediction:", np.argmax(prediction))